In [2]:
#!/usr/bin/env python
# cofing: utf-8

# from retry import retry
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
from tqdm import tqdm
import re
import math
# from logging import getLogger, StreamHandler, Formatter, FileHandler, DEBUG
import yaml
import os
from retry import retry
import sys
import time

notebook = True
if notebook:
    work_dir = '/Users/satomitakei/property_valuation_calculator'
    with open(work_dir + '/setting/kenbiya_scraping_config.yaml', 'r') as yml:
        config = yaml.safe_load(yml)
    area_name = 'tokyo'
else:
    work_dir = os.getcwd()
    with open(work_dir + '/setting/kenbiya_scraping_config.yaml', 'r') as yml:
        config = yaml.safe_load(yml)
    area_name = sys.argv[1]
# if area_name == 'tokyo':
#     base_url = config['base_url_tokyo']
# elif area_name == 'osaka':
#     base_url = config['base_url_osaka']
# elif area_name == 'fukuoka':
#     base_url = config['base_url_fukuoka']

def write_log(log_file, text):
    f = open(log_file, 'a', encoding='UTF-8')
    # f.write(text)
    # f.close()
    print(text)

diff_jst_from_utc = 0
start_time = dt.datetime.now() + dt.timedelta(hours=diff_jst_from_utc)
now_time = (dt.datetime.now() +
            dt.timedelta(hours=diff_jst_from_utc)).strftime('%Y%m%d_%H%M')

log_dir = work_dir + f'/log/scraping'
os.makedirs(log_dir, exist_ok=True)
log_file = log_dir + f'/{now_time}_log.txt'
f = open(log_file, 'w', encoding='UTF-8')
f.close()

text = 'processing_start_time:' + str(start_time.replace(microsecond=0)) + '\n'
write_log(log_file, text)
excution_date = dt.datetime.today().strftime('%Y%m%d')

# file_name = 'suumo_baibai'
# excution_date = dt.datetime.today().strftime('%Y%m%d')
base_url = 'https://www.rakumachi.jp/syuuekibukken/area/prefecture/dimAll/?area%5B%5D=13101&area%5B%5D=13102&area%5B%5D=13103&area%5B%5D=13104&area%5B%5D=13105&area%5B%5D=13106&area%5B%5D=13107&area%5B%5D=13108&area%5B%5D=13109&area%5B%5D=13110&area%5B%5D=13111&area%5B%5D=13112&area%5B%5D=13113&area%5B%5D=13114&area%5B%5D=13115&area%5B%5D=13116&area%5B%5D=13117&area%5B%5D=13118&area%5B%5D=13119&area%5B%5D=13120&area%5B%5D=13121&area%5B%5D=13122&area%5B%5D=13123&newly=&price_from=&price_to=&gross_from=&gross_to=&year_from=&year_to=&b_area_from=&b_area_to=&houses_ge=&houses_le=&min=&l_area_from=&l_area_to=&keyword=&floor_unit_price_from=&floor_unit_price_to=&land_unit_price_from=&land_unit_price_to='
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup
@retry(tries=3, delay=10, backoff=2)
def main():
    all_data = []

    # 基本ページurl 
    page = '1'
    url = base_url.format(page = page)
    write_log(log_file,'base_url:'+url)
    # get html
    item = get_html(url)

    # extract all items
    total_rooms = int(re.sub(r"\D", "",item.find(True,"strong", class_="result_num").get_text()))
    max_page = math.floor(total_rooms/50)+ 1
    text = f"max_page:{max_page} \n"
    write_log(log_file,text)

    url_list = []
    all_data = []
    error_page = []
    # 物件URLの取得
    data= {}
    # for i in tqdm(range(max_page+1)): 
    for i in tqdm(range(2)): 
        url = base_url.format(page = str(i))
        # get html
        item = get_html(url)
        for j in item.findAll(href=re.compile("pp1/s/.*/re_.*/")):
            print(j.get('href'))
            room_url = 'https://www.kenbiya.com/'+j.get('href')
            write_log(log_file,'room_url'+room_url)
            print(room_url)
            try:
                room_item = get_html(room_url)
                # 物件詳細のデータを収集
                print(room_item)
                data["mansion_name"] = room_item.find("h1", {"class": "inner ttl"}).getText().strip()
                print('mansion_name:'+data["mansion_name"])
                write_log(log_file,data["mansion_name"])
                time.sleep(1)
                print('room done')
            except:
                write_log(log_file,'error')
            #     data["price"] = room_item.find("p", {"class": "mt7 b"}).getText().strip()
            #     data["floor_plan"] = room_item.findAll("td", {"class": "w290 bdCell"})[1].getText().strip()
            #     data["total_rooms"] = room_item.findAll("td", {"class": "w290 bdCell"})[3].getText().strip()
            #     data["exclusive_area"] = room_item.findAll("td", {"class": "w290 bdCell"})[4].getText().strip()
            #     data["other_area"] = room_item.findAll("td", {"class": "w290 bdCell"})[5].getText().strip()
            #     data["stories"] = room_item.findAll("td", {"class": "w290 bdCell"})[6].getText().strip()
            #     data["completion"] = room_item.findAll("td", {"class": "w290 bdCell"})[7].getText().strip()
            #     data["adress"] = room_item.findAll("td", {"class": "w290 bdCell"})[8].getText().strip()
            #     data["access"] = room_item.findAll("td", {"class": "w290 bdCell"})[9].getText().strip()

            #     # 物件詳細概要ページからのデータを取得
            #     room_detail_item = get_html(room_detail_url)
            #     data["move_in_date"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[12].getText().strip()
            #     data["direction"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[15].getText().strip()
            #     data["reform"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[16].getText().strip()
            #     data["ownership"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[23].getText().strip()
            #     data["use_district"] = room_detail_item.findAll("td", {"class": "w299 bdCell"})[24].getText().strip()
            #     data["url"] = room_url
            #     data["log_date"] = excution_date
            #     all_data.append(data)
            # except:
            #     error_page.append(room_url)

        df = pd.DataFrame(data)
        df.to_csv(work_dir+f'/scraping_raw/kenbiya_baibai_{excution_date}.csv',index = False)
        
    text = 'df_shape:{}\n'.format(df.shape)
    write_log(log_file,text)

    end_time = dt.datetime.now() + dt.timedelta(hours=diff_jst_from_utc)
    text = 'predicting done.\nend_time:{}\n'.format(end_time)
    write_log(log_file,text)

    processing_time = end_time - start_time
    text = 'processing_time:{}\n'.format(processing_time)
    write_log(log_file,text)

if __name__ == '__main__':
    main()


processing_start_time:2023-02-23 16:24:24

base_url:https://www.rakumachi.jp/syuuekibukken/area/prefecture/dimAll/?area%5B%5D=13101&area%5B%5D=13102&area%5B%5D=13103&area%5B%5D=13104&area%5B%5D=13105&area%5B%5D=13106&area%5B%5D=13107&area%5B%5D=13108&area%5B%5D=13109&area%5B%5D=13110&area%5B%5D=13111&area%5B%5D=13112&area%5B%5D=13113&area%5B%5D=13114&area%5B%5D=13115&area%5B%5D=13116&area%5B%5D=13117&area%5B%5D=13118&area%5B%5D=13119&area%5B%5D=13120&area%5B%5D=13121&area%5B%5D=13122&area%5B%5D=13123&newly=&price_from=&price_to=&gross_from=&gross_to=&year_from=&year_to=&b_area_from=&b_area_to=&houses_ge=&houses_le=&min=&l_area_from=&l_area_to=&keyword=&floor_unit_price_from=&floor_unit_price_to=&land_unit_price_from=&land_unit_price_to=
base_url:https://www.rakumachi.jp/syuuekibukken/area/prefecture/dimAll/?area%5B%5D=13101&area%5B%5D=13102&area%5B%5D=13103&area%5B%5D=13104&area%5B%5D=13105&area%5B%5D=13106&area%5B%5D=13107&area%5B%5D=13108&area%5B%5D=13109&area%5B%5D=13110&area%5B%5D=

AttributeError: 'NoneType' object has no attribute 'get_text'

In [5]:
url

NameError: name 'url' is not defined

In [6]:
item = get_html(base_url)
item

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="width=device-width, user-scalable=no, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<title>楽待</title>
<style>body{font-family:Meiryo,sans-serif}</style>
</meta></head>
<body>
<div style="width:100%; margin:0 auto; background-color:#FFF; padding:6% 0;">
<h1 style="text-align:center; color:#727171;">
<img src="data:image/png;base64,R0lGODlhggBJAPcAAO/v7+7u7u3t7ezs7Orq6unp6ejo6Ofn5+bm5uXl5eTk5OPj4+Li4uHh4d/f
397e3tzc3Nvb29ra2tnZ2djY2NfX19bW1tXV1dTU1NPT09LS0tHR0c/Pz87Ozs3NzczMzMvLy8rK
ysnJycjIyMfHx8bGxsXFxcTExMLCwsHBwcDAwL+/v76+vr29vby8vLu7u7q6urm5ubi4uLe3t7a2
trS0tLOzs7KysrGxsbCwsK+vr66urq2traysrKurq6qqqqmpqaenp6ampqWlpaSkpKOjo6KioqGh
oaCgoJ+fn56enp2dnZycnJubm5mZmZiYmJeXl5aWlpWVlZSUlJOTk5KSkpCQkI+Pj46OjoyMjIuL
i4qKiomJiYiIiIeHh4aG

In [77]:
item.findAll(href=re.compile("pp1/s/.*/re_.*/"))[0].get('href')

'/pp1/s/tokyo/nerima-ku/re_2956389ruc/'

In [71]:
for i in item.findAll(href=re.compile("pp1/s/.*/re_.*/")):
    print(i.get('href'))

/pp1/s/tokyo/nerima-ku/re_2956389ruc/
/pp1/s/tokyo/kita-ku/re_29563791hs/
/pp1/s/tokyo/meguro-ku/re_2956378uq5/
/pp1/s/tokyo/chiyoda-ku/re_2956377fp7/
/pp1/s/tokyo/kita-ku/re_2956348ghm/
/pp1/s/tokyo/setagaya-ku/re_2956335jef/
/pp1/s/tokyo/nerima-ku/re_2956324suu/
/pp1/s/tokyo/minato-ku/re_2956288axf/
/pp1/s/tokyo/minato-ku/re_2956286vjc/
/pp1/s/tokyo/toshima-ku/re_2956285z2p/
/pp1/s/tokyo/taito-ku/re_2956284y7a/
/pp1/s/tokyo/sumida-ku/re_2956272jzh/
/pp1/s/tokyo/itabashi-ku/re_2956271boh/
/pp1/s/tokyo/ota-ku/re_2956269msu/
/pp1/s/tokyo/ota-ku/re_2956265rjy/
/pp1/s/tokyo/suginami-ku/re_2956263sgq/
/pp1/s/tokyo/shibuya-ku/re_2956262jwv/
/pp1/s/tokyo/bunkyo-ku/re_2956261us0/
/pp1/s/tokyo/shinjuku-ku/re_29562602us/
/pp1/s/tokyo/itabashi-ku/re_2956259izq/
/pp1/s/tokyo/ota-ku/re_2956258mpx/
/pp1/s/tokyo/bunkyo-ku/re_2956249uek/
/pp1/s/tokyo/shinjuku-ku/re_2956240xjo/
/pp1/s/tokyo/nerima-ku/re_2956232un7/
/pp1/s/tokyo/minato-ku/re_2956231xzo/
/pp1/s/tokyo/itabashi-ku/re_29562303yx/
/pp1/s/to

In [57]:
text = item.findAll(href=re.compile("pp1/s/.*/re_.*/"))[0].get('href')
# get('href')
text

'/pp1/s/tokyo/nerima-ku/re_2956389ruc/'

In [54]:
text

[<a href="/pp1/s/tokyo/nerima-ku/re_2956389ruc/" target="_blank">
 <ul class="prop_block">
 <li class="photo">
 <input data-closed_flg="N" id="ck_1" name="ck_pp" type="checkbox" value="2956389ruc"/><label for="ck_1">
 <img alt="不動産投資の投資用マンション" class="cate_icon" loading="lazy" src="/images/prop/cate_ONE.svg"/>
 <p><img alt="東京都練馬区の投資用マンション／豊島園_画像" loading="lazy" src="/upload/p2956/2956389/1661218999700-haj_wm_3229_000.jpg"/></p>
 </label>
 </li>
 <li class="main">
 <ul>
 <li><h3>オメガエステートよりお届け！「練馬」駅 徒歩3分！！</h3></li>
 <li>東京都練馬区豊玉北4-14-8</li>
 <li>都営大江戸線 豊島園駅 歩3分</li>
 <li><span class="new">New</span></li>
 </ul>
 </li>
 <li class="price">
 <ul>
 <li><span>730</span>万円</li>
 <li><span>7<span>.72</span></span>％</li>
 </ul>
 </li>
 <li>
 <ul>
 <li>専:15.00m²</li>
 </ul>
 </li>
 <li>
 <ul>
 <li>1985年1月</li>
 <li>1階/4階建</li>
 </ul>
 </li>
 </ul>
 </a>,
 <a href="/pp1/s/tokyo/kita-ku/re_29563791hs/" target="_blank">
 <ul class="prop_block">
 <li class="photo">
 <input data-closed_flg="N" id="ck